In [1]:
import pandas as pd

In [7]:
# 🔹 Chemins
input_path = "../../data/cleaning/step1_typed.parquet"

# 🔹 Chargement des données
df = pd.read_parquet(input_path)

# Définir les variables à partir des colonnes du DataFrame
M = df['loan_amnt']
t_pourcentage = df['int_rate']
n = df['term']

# Convertir le taux d'intérêt de pourcentage à décimal et en taux mensuel
t = (t_pourcentage / 100) / 12

# Calculer le montant de chaque mensualité (m)
m = (M * t) / (1 - (1 + t)**(-n))

# Ajouter la colonne des mensualités calculées au DataFrame
df['monthly_payment_calculated'] = m

# Diviser la colonne 'installment' par 10
df['installment_divided'] = df['installment'] / 10

# Calculer l'écart en pourcentage entre les deux colonnes
df['percentage_difference'] = ((df['monthly_payment_calculated'] - df['installment_divided']) / df['installment_divided']) * 100

# Afficher les deux colonnes finales côte à côte
print("Comparaison des mensualités :")
print(df[['installment_divided', 'monthly_payment_calculated','percentage_difference']])

# Calculer et afficher les statistiques de l'écart en pourcentage
max_difference = df['percentage_difference'].max()
min_difference = df['percentage_difference'].min()
mean_difference = df['percentage_difference'].mean()
std_difference = df['percentage_difference'].std()

print("\nStatistiques de l'écart en pourcentage :")
print(f"Maximum d'écart : {max_difference:.2f}%")
print(f"Minimum d'écart : {min_difference:.2f}%")
print(f"Moyenne d'écart : {mean_difference:.2f}%")
print(f"Écart type de l'écart : {std_difference:.2f}%")

Comparaison des mensualités :
        installment_divided  monthly_payment_calculated  percentage_difference
0                    3294.8                 3294.747814              -0.001584
1                    2656.8                 2656.762705              -0.001404
2                    5069.7                 5069.645631              -0.001072
3                    2206.5                 2206.400507              -0.004509
4                    6093.3                 6093.258624              -0.000679
...                     ...                         ...                    ...
396025               2173.8                 2173.743632              -0.002593
396026               7004.2                 7004.127849              -0.001030
396027               1613.2                 1613.124623              -0.004673
396028               5030.2                 5030.122114              -0.001548
396029                679.8                  679.770326              -0.004365

[396030 rows x 3 colu

In [9]:
# Définir un seuil pour identifier les grands écarts (vous pouvez ajuster ce seuil)
threshold_large_percentage = 5  # Exemple : écart de plus de 5% en valeur absolue

# Isoler les lignes où l'écart en pourcentage dépasse le seuil de grand écart
grand_ecart_rows = df[abs(df['percentage_difference']) > threshold_large_percentage]

# Afficher les lignes avec un grand écart en pourcentage
print(f"Lignes avec un écart en pourcentage supérieur à {threshold_large_percentage}% en valeur absolue :")
print(grand_ecart_rows[['installment', 'installment_divided', 'monthly_payment_calculated', 'percentage_difference']])

# Vous pouvez également afficher le nombre de ces lignes
print("\nNombre de lignes avec un grand écart :", len(grand_ecart_rows))

Lignes avec un écart en pourcentage supérieur à 5% en valeur absolue :
        installment  installment_divided  monthly_payment_calculated  \
9              9284                928.4                 9283.998464   
12             2391                239.1                 2390.918392   
13             7837                783.7                 7836.945052   
15             5673                567.3                 5672.986514   
18           106387              10638.7                 9559.911488   
...             ...                  ...                         ...   
396007        52588               5258.8                 4268.938703   
396008         4261                426.1                 4260.974529   
396010        31754               3175.4                 2816.056034   
396012         8638                863.8                 6722.801867   
396015         1275                127.5                 1274.970041   

        percentage_difference  
9                  899.999835  
